# Train a regressor on a set of embeddings of species texts
---"ratinale", "habitat", "threat", "population" 

In [2]:
import pandas as pd
ass = pd.read_csv('/Users/julie/Desktop/labs/project-canopy-ds/data/clean_data/assessments.csv',index_col = False)

In [4]:
ass.head()

,assessmentId,internalTaxonId,scientificName,redlistCategory,redlistCriteria,yearPublished,assessmentDate,criteriaVersion,language,rationale,...,populationTrend,range,useTrade,systems,conservationActions,realm,yearLastSeen,possiblyExtinct,possiblyExtinctInTheWild,scopes
0,519491,137074,Paraphytus africanus,Data Deficient,Unknown,2013,2013-07-30 00:00:00 UTC,3.1,English,Paraphytus africanus has been listed as Data D...,...,Unknown,This species was described jointly from Angol...,There is no use or trade information availabl...,Terrestrial,There are no species-specific conservation ac...,Afrotropical,Unknown,False,False,Global
1,520806,137161,Eodrepanus fastiditus,Least Concern,Unknown,2013,2013-07-30 00:00:00 UTC,3.1,English,As Eodrepanus fastiditus is a very wide rangin...,...,Unknown,This species was described from Frere in KwaZ...,There is no use or trade information available...,Terrestrial,There are no species-specific conservation me...,Afrotropical,Unknown,False,False,Global
2,521021,137173,Onthophagus lacustris,Least Concern,Unknown,2013,2013-07-30 00:00:00 UTC,3.1,English,As Onthophagus lacustris is found in shaded s...,...,Unknown,"For the purpose of this assessment, we restri...",Specimens of this species are offered for sal...,Terrestrial,There are no species-specific conservation ac...,Afrotropical,Unknown,False,False,Global
3,521177,137186,Onthophagus pugionatus,Least Concern,Unknown,2013,2013-07-30 00:00:00 UTC,3.1,English,Although Onthophagus pugionatus shows a large ...,...,Unknown,This species was described from unspecified l...,There is no use or trade information available...,Terrestrial,There are no species-specific conservation ac...,Afrotropical,Unknown,False,False,Global
4,522595,137281,Pedaria ovata,Data Deficient,Unknown,2014,2014-03-06 00:00:00 UTC,3.1,English,Pedaria ovata has been assessed as Data Defici...,...,Unknown,This species is known from Democratic Republic...,This species is unlikely to be utilized.,Terrestrial,There are no species-specific conservation mea...,Afrotropical,Unknown,False,False,Global


In [19]:
ass_new = ass[["rationale","habitat","threats","population","populationTrend","range","useTrade","conservationActions"]]

In [20]:
ass_new.head()

,rationale,habitat,threats,population,populationTrend,range,useTrade,conservationActions
0,Paraphytus africanus has been listed as Data D...,There are no habitat data available for this ...,Considering that this species probably shares...,There are no quantitative population data ava...,Unknown,This species was described jointly from Angol...,There is no use or trade information availabl...,There are no species-specific conservation ac...
1,As Eodrepanus fastiditus is a very wide rangin...,Locality records reported for this very sma...,As this species is found in various dung types...,This species is recorded with a relatively low...,Unknown,This species was described from Frere in KwaZ...,There is no use or trade information available...,There are no species-specific conservation me...
2,As Onthophagus lacustris is found in shaded s...,"In Mkuzi Game Reserve in South Africa, trappin...",Clearance of forest or shaded thickets is lik...,This species is locally abundant in Maputo El...,Unknown,"For the purpose of this assessment, we restri...",Specimens of this species are offered for sal...,There are no species-specific conservation ac...
3,Although Onthophagus pugionatus shows a large ...,There is ample evidence that this very small-...,As this species is associated with shaded veg...,Quantitative data suggest that this species o...,Unknown,This species was described from unspecified l...,There is no use or trade information available...,There are no species-specific conservation ac...
4,Pedaria ovata has been assessed as Data Defici...,The natural history of this species is poorly ...,The threats to this species are unknown. Altho...,There is no population information available f...,Unknown,This species is known from Democratic Republic...,This species is unlikely to be utilized.,There are no species-specific conservation mea...


In [26]:
ass['combined']= ass_new.values

In [27]:
# we only need the texts for making embeddings
test_texts = [text for text in ass['combined']]
len(test_texts)

6622

In [29]:
#test_texts

In [31]:
!pip install basilica 

In [35]:
import numpy as np
import sklearn.linear_model
import sklearn.preprocessing
import sklearn.decomposition
import basilica

In [43]:
#API_KEY = 
# get a real one
API_KEY = '53d4be4d-9eef-19ef-343c-18a210aeb81e'


# volume of use of this is monitored by basilica, especially when usinf 'SLOW_DEMO_KEY'
# uncomment and run only when needed


with basilica.Connection(API_KEY) as c:
    embeddings = list(c.embed_sentences(test_texts))
    
len(embeddings)

6622

In [44]:
# Normalize to help regressor work better
normalized_embeddings = sklearn.preprocessing.normalize(embeddings)
len(normalized_embeddings)
# normalized_embeddings[0]

# put the normalized embeddings back in a dataframe
normalized_embeddings_df = pd.DataFrame(normalized_embeddings)
# PCA the embeddings
# sklearn.get_config()
normalized_embeddings_df.shape

(6622, 768)

# Split into X matrix (embeddings) and y vector (redlistCategory)

In [45]:
X = normalized_embeddings_df

y =ass['redlistCategory']
print(y.shape)
print(y.isna().value_counts())


(6622,)
False    6622
Name: redlistCategory, dtype: int64


In [46]:
print(X.shape)

(6622, 768)


# Training a regressor

In [47]:
X_train, X_test = sklearn.model_selection.train_test_split(X, random_state=72)
y_train, y_test = sklearn.model_selection.train_test_split(y, random_state=72)

model = sklearn.linear_model.LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

/Users/julie/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/julie/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
print('Train accuracy: %.3f' % model.score(X_train, y_train))
print('Test accuracy: %.3f' % model.score(X_test, y_test))

Train accuracy: 0.831
Test accuracy: 0.853


In [52]:
def predict_one_embedding(embdng):
    """Use model to predict based on one embedding."""
    return ('predicted redlist category', model.predict([embdng])[0])

def predict_one_by_index(embeddings, idx):
    """Use model to predict based on one embedding,
    selected by index from a list of embeddings."""
    return (predict_one_embedding(embeddings[idx]), 'actual', y[idx])


print(predict_one_by_index(embeddings,2))


(('predicted redlist category', 'Least Concern'), 'actual', 'Least Concern')


In [53]:
print(predict_one_by_index(embeddings,10))

(('predicted redlist category', 'Near Threatened'), 'actual', 'Data Deficient')


In [54]:
print(predict_one_by_index(embeddings,100))

(('predicted redlist category', 'Least Concern'), 'actual', 'Least Concern')


In [55]:
print(predict_one_by_index(embeddings,25))

(('predicted redlist category', 'Near Threatened'), 'actual', 'Near Threatened')


In [84]:
predict = model.predict(X_train)
predict

array(['Least Concern', 'Data Deficient', 'Least Concern', ...,
       'Vulnerable', 'Least Concern', 'Endangered'], dtype=object)

In [85]:
pred = pd.DataFrame(predict)

In [86]:
# make the index of X_test to an own dataframe
pre_index = pd.DataFrame(X_train.index)
pre_index.columns = ['pre_index']

# ... and concat this dataframe with y_test
pred = pd.concat([pred, pre_index], axis=1)
pred.set_index(['pre_index'], inplace=True)

In [88]:
pred

,0
pre_index,
4492,Least Concern
523,Data Deficient
4254,Least Concern
4453,Least Concern
1905,Critically Endangered
...,...
2885,Least Concern
3941,Least Concern
5166,Vulnerable


In [89]:
y_train

4492            Least Concern
523            Data Deficient
4254            Least Concern
4453            Least Concern
1905    Critically Endangered
                ...          
2885            Least Concern
3941            Least Concern
5166               Vulnerable
5907            Least Concern
4568    Critically Endangered
Name: redlistCategory, Length: 4966, dtype: object

In [92]:
result = pd.concat([y_train, pred], axis=1)

In [94]:
result.rename(columns={0:'predict'},inplace = True)

In [96]:
result.head(100)

,redlistCategory,predict
4492,Least Concern,Least Concern
523,Data Deficient,Data Deficient
4254,Least Concern,Least Concern
4453,Least Concern,Least Concern
1905,Critically Endangered,Critically Endangered
...,...,...
1826,Least Concern,Least Concern
5603,Least Concern,Least Concern
2436,Least Concern,Least Concern
4560,Least Concern,Least Concern


In [3]:
ass['habitat']

0       There are no habitat data  available for this ...
1         Locality records reported for  this very sma...
2       In Mkuzi Game Reserve in South Africa, trappin...
3       There is ample evidence that  this very small-...
4       The natural history of this species is poorly ...
                              ...                        
6617    Behaviour This is an Afrotropical species (15°...
6618                                              Unknown
6619                                              Unknown
6620    Habitat Types and RangeLeopards occur in the w...
6621    This species is found in primary and secondary...
Name: habitat, Length: 6622, dtype: object

In [53]:
ass['habitat'][0]

'There are no habitat data  available for this species because locality records are all too inexact to make  useful deductions. However, Paraphytus species are  known to be restricted to tropical rainforests where one species (Paraphytus aphodioides Boucomont  1928) has been shown to breed within rotting logs using the debris and digested  wood droppings from other insects to construct a brood within which an egg is  laid (Cambefort and Walter 1985). This brood is tended  by the female. Such specialized habits probably explain the rarity of the  African species in collections. Owing to the probable inaccuracy of the locality  data, the coincidence with ecoregions is provided only  for two of the records. Unfortunately, neither of these is that of the type  locality.  The Arebi record coincides with  the edge of the Northeastern Congolian lowland forests  (AT0124) ecoregion and the island of Säo Tomé entirely comprises part of the Säo Tomé and Príncipe moist lowland forests (AT0127) eco

In [52]:
filter_col = [col for col in ass['habitat'][0].split() if col.startswith('T')]
filter_col

['There', 'This', 'The', 'Tomé', 'Tomé', 'The']

In [54]:
list1 = []
for i in ass['habitat']:
    filter_col = [col for col in i.split() if col.startswith('T')]
    list1.append(filter_col)

In [55]:
list1 = pd.DataFrame(list1)

In [56]:
list1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,There,This,The,Tomé,Tomé,The,None,None,None,None,None,None,None,None,None,None,None,None
1,The,Tshwane,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,The,Tanzania,The,This,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,There,Tunya),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,The,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6617,This,The,The,The,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6618,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6619,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6620,Types,They,There,They,They,They,Though,These,The,The,"Thailand,",None,None,None,None,None,None,None


In [57]:
l1 = ass['habitat'].apply(lambda x:[col for col in x.split() if col.startswith('T')])

In [58]:
l1

0                     [There, This, The, Tomé, Tomé, The]
1                                          [The, Tshwane]
2                              [The, Tanzania, The, This]
3                                         [There, Tunya)]
4                                                   [The]
                              ...                        
6617                                [This, The, The, The]
6618                                                   []
6619                                                   []
6620    [Types, They, There, They, They, They, Though,...
6621                       [This, They, They, They, This]
Name: habitat, Length: 6622, dtype: object

In [59]:
l1 = l1.apply(lambda x:','.join(map(str,x)))

In [60]:
l1

0                            There,This,The,Tomé,Tomé,The
1                                             The,Tshwane
2                                   The,Tanzania,The,This
3                                            There,Tunya)
4                                                     The
                              ...                        
6617                                     This,The,The,The
6618                                                     
6619                                                     
6620    Types,They,There,They,They,They,Though,These,T...
6621                             This,They,They,They,This
Name: habitat, Length: 6622, dtype: object